In [37]:
import cv2
import numpy as np
from insightface.app import FaceAnalysis
from scipy.spatial import procrustes
from PIL import Image, ImageDraw, ImageFont

In [38]:
def process_image_for_faces(image_path):
    app = FaceAnalysis(allowed_modules=["detection", "landmark_2d_106"], providers=["CUDAExecutionProvider"])
    app.prepare(ctx_id=0, det_size=(640, 640))

    img = cv2.imread(image_path)
    if img is None:
        raise ValueError("Image not found or unable to load.")
    
    faces_data = app.get(img)
    faces_landmarks = []
    bounding_boxes = []
    
    for face in faces_data:
        if "landmark_2d_106" in face:
            lmk = face["landmark_2d_106"]
            bbox = face["bbox"].astype(np.int32)
            faces_landmarks.append(np.round(lmk).astype(np.int64))
            bounding_boxes.append(bbox)
    
    return faces_landmarks, bounding_boxes, img

In [39]:
def compare_faces_landmarks(landmarks1, landmarks2):
    results = []
    print("Procrustes disparity results for all possible face matches:")
    for idx1, lmk1 in enumerate(landmarks1):
        for idx2, lmk2 in enumerate(landmarks2):
            mtx1, mtx2, disparity = procrustes(lmk1, lmk2)
            results.append((idx1, idx2, disparity))
            print(f"Face {idx1} in Image 1 vs Face {idx2} in Image 2: Disparity = {disparity:.4f}")
    return results

In [41]:
def draw_matching_faces(image1, landmarks1, bounding_boxes1, image2, landmarks2, bounding_boxes2, matches):
    pil_image1 = Image.fromarray(cv2.cvtColor(image1, cv2.COLOR_BGR2RGB))
    pil_image2 = Image.fromarray(cv2.cvtColor(image2, cv2.COLOR_BGR2RGB))
    draw1 = ImageDraw.Draw(pil_image1)
    draw2 = ImageDraw.Draw(pil_image2)

    # Load a font
    try:
        font = ImageFont.truetype("arial.ttf", 16)
    except IOError:
        font = ImageFont.load_default()

    color = (255, 0, 0)  # Set a color for the bounding box and landmarks
    point_radius = 2
    
    for idx1, (landmarks, bbox) in enumerate(zip(landmarks1, bounding_boxes1)):
        for point in landmarks:
            draw1.ellipse([point[0]-point_radius, point[1]-point_radius, point[0]+point_radius, point[1]+point_radius], fill=color)
        draw1.rectangle([bbox[0], bbox[1], bbox[2], bbox[3]], outline=color, width=2)
        draw1.text((bbox[0], bbox[1] - 20), f"Face {idx1}", fill=(255, 255, 255), font=font)
        
    for idx2, (landmarks, bbox) in enumerate(zip(landmarks2, bounding_boxes2)):
        for point in landmarks:
            draw2.ellipse([point[0]-point_radius, point[1]-point_radius, point[0]+point_radius, point[1]+point_radius], fill=color)
        draw2.rectangle([bbox[0], bbox[1], bbox[2], bbox[3]], outline=color, width=2)
        draw2.text((bbox[0], bbox[1] - 20), f"Face {idx2}", fill=(255, 255, 255), font=font)

    # Highlight matching faces with a different color
    match_color = (0, 255, 0)
    for match in matches:
        idx1, idx2, disparity = match
        if disparity < 0.05:  # Threshold for matching, can be adjusted
            bbox1 = bounding_boxes1[idx1]
            bbox2 = bounding_boxes2[idx2]
            draw1.rectangle([bbox1[0], bbox1[1], bbox1[2], bbox1[3]], outline=match_color, width=3)
            draw2.rectangle([bbox2[0], bbox2[1], bbox2[2], bbox2[3]], outline=match_color, width=3)

    pil_image1.show()
    pil_image2.show()

In [46]:
# Paths to the images
img_path1 = "data/aespa4.jpg"
img_path2 = "data/aespa5.jpg"

landmarks1, boxes1, img1 = process_image_for_faces(img_path1)
landmarks2, boxes2, img2 = process_image_for_faces(img_path2)

if landmarks1 and landmarks2:
    comparison_results = compare_faces_landmarks(landmarks1, landmarks2)
    draw_matching_faces(img1, landmarks1, boxes1, img2, landmarks2, boxes2, comparison_results)
else:
    print("Landmarks were not detected in one or both images.")

Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
model ignore: C:\Users\hancomtst/.insightface\models\buffalo_l\1k3d68.onnx landmark_3d_68
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\hancomtst/.insightface\models\buffalo_l\2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\hancomtst/.insightface\models\buffalo_l\det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
model ignore: C:\Users\hancomtst/.insightface\models\buffalo_l\genderage.onnx genderage
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
model ignore: C:\Users\hancomtst/.insightface\models\buffalo_l\w600k_r50.onnx recognition
set det-size: (640, 640)
Applied providers: ['CPUExecution